In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import pickle
import numpy as np

In [2]:
with open('train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
with open('test_qa.txt', 'rb') as f:
    test_data = pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(train_data)

10000

In [7]:
len(test_data)

1000

In [8]:
# Story
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
# Question
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [10]:
# Answer
train_data[0][2]

'no'

In [11]:
all_data = test_data + train_data

In [12]:
vocab = set()

for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [13]:
vocab.add('no')

In [14]:
vocab.add('yes')

In [15]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [16]:
vocab_len = len(vocab) + 1

In [17]:
vocab_len

38

In [18]:
# Longest story
all_story_lens = [len(data[0]) for data in all_data]

In [19]:
max_story_len = max(all_story_lens)

In [20]:
max_story_len

156

In [21]:
max_question_len = max([len(data[1]) for data in all_data ])

In [22]:
max_question_len

6

In [23]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [24]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(vocab)

In [25]:
tokenizer.word_index

{'john': 1,
 'hallway': 2,
 'picked': 3,
 'bathroom': 4,
 'took': 5,
 'yes': 6,
 'put': 7,
 'up': 8,
 'the': 9,
 'bedroom': 10,
 '?': 11,
 'no': 12,
 'milk': 13,
 'kitchen': 14,
 'moved': 15,
 '.': 16,
 'down': 17,
 'daniel': 18,
 'discarded': 19,
 'got': 20,
 'football': 21,
 'office': 22,
 'went': 23,
 'sandra': 24,
 'in': 25,
 'garden': 26,
 'grabbed': 27,
 'left': 28,
 'is': 29,
 'dropped': 30,
 'back': 31,
 'to': 32,
 'journeyed': 33,
 'mary': 34,
 'travelled': 35,
 'apple': 36,
 'there': 37}

In [26]:
train_story_text = []
train_question_text = []
train_answers = []

In [27]:
for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [28]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [29]:
len(train_story_seq)

10000

In [30]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    # Stories
    X = []
    # Questions
    Xq = []
    # Y Correct answer
    Y = []
    
    for story, question, answer in data:
        # for each story
        # [23, 14....]
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in question]
        
        y = np.zeros(len(word_index) + 1)
        
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))
        

In [31]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [32]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [33]:
from keras.models import Sequential, Model

In [34]:
from keras.layers.embeddings import Embedding

In [35]:
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [36]:
# Placeholder shape=(max_story_len, batch_size)
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [37]:
vocab_size = len(vocab) + 1

In [38]:
# Input encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_m.add(Dropout(0.3))

# Outputs
# (Samples, story max len, embedding dimension)

In [39]:
# Input encoder C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

# Outputs
# (Samples, story max len, max_question_len)

In [40]:
# Question encoder
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_question_len))
question_encoder.add(Dropout(0.3))

# Outputs
# (Samples, questions max len, embedding dimension)

In [41]:
# ENCODED <---- ECONDER(INPUT)
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [42]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [43]:
response = add([match, input_encoded_c])
response = Permute((2, 1))(response)

In [44]:
answer = concatenate([response, question_encoded])

In [45]:
answer

<tf.Tensor 'concatenate_1/concat:0' shape=(None, 6, 220) dtype=float32>

In [46]:
answer = LSTM(32)(answer)

In [47]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)

In [48]:
answer = Activation('softmax')(answer)

In [49]:
model = Model([input_sequence, question], answer)

In [50]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 156)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
____________________________________________________________________________________________

In [52]:
# history = model.fit(
#     [inputs_train, queries_train],
#     answers_train,
#     batch_size=32,
#     epochs=3,
#     validation_data=([inputs_test, queries_test], answers_test)
# )

In [53]:
model.load_weights('chatbot_10.h5')

In [54]:
pred_results = model.predict(([inputs_test, queries_test]))

In [55]:
val_max = np.argmax(pred_results[0])

In [56]:
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [57]:
k

'yes'

In [58]:
pred_results[0][val_max]

0.9367328

In [59]:
my_story = "John left the kitchen . Sandra dropped the football in the garden ."

In [60]:
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [61]:
my_question = "Is the football in the garden ?"

In [62]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [65]:
mydata = [(my_story.split(), my_question.split(), 'yes')]

In [66]:
my_story, my_quest, my_ans = vectorize_stories(mydata)

In [67]:
pred_results = model.predict(([my_story, my_quest]))

In [68]:
val_max = np.argmax(pred_results[0])

In [69]:
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [70]:
k

'yes'